In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
# https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
a=os.getcwd()
ROOT_PATH=os.path.join(a,"gdrive","My Drive","python-cloud")


In [3]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Embedding, GRU, LSTM, MaxPooling2D, GlobalMaxPool1D, CuDNNGRU, CuDNNLSTM, Reshape
from keras.layers import Dropout, Dense, Activation, Flatten,Conv2D, Bidirectional, SpatialDropout1D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, ZeroPadding2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        
        model.add(Dense(300))
        model.add(Dropout(0.5))
        model.add(Dense(225))
        model.add(Dropout(0.5))
        model.add(Reshape((15, 15)))
        model.add(CuDNNGRU(86))
        
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
          
          #valid means zero padding

#           model = Sequential()
#           model.add(Conv2D(1, kernel_size=(1, 1), strides=1, activation='relu', input_shape=(28,28,1)))
#           model.add(BatchNormalization())
#           model.add(Conv2D(29, 5, padding='valid', strides=1, activation='relu'))
#           model.add(BatchNormalization())
#           model.add(MaxPooling2D(3,  padding='same',strides=2))

#           model.add(Conv2D(59, 3,  padding='valid',strides=1, activation='relu')) # increase kernel size to 5
#           model.add(BatchNormalization())
#           model.add(MaxPooling2D(3,strides=2,  padding='same'))

#           model.add(Conv2D(74, 3, strides=1, activation='relu'))
#           model.add(BatchNormalization())

#           model.add(MaxPooling2D(3,  padding='same',strides=2)) 
#           model.add(Flatten())
#           model.add(Dense(300))
#           model.add(Dropout(0.5))
#           model.add(Dense(300))
#           model.add(Dropout(0.5))

#model.add(Dense(32*32))

#model.add(Reshape((15, 15)))
#model.add(LSTM(100, dropout=0.1, recurrent_dropout=0.5))
# let's load the weights from a save file.
#image_model.load_weights('weight_file.h5')
#model.add(GRU(86, dropout=0.1, recurrent_dropout=0.5)) # defaults inclide tanh activation
#model.add(CuDNNGRU(86))
#           model.add(Dense(1, activation='softmax'))
#           model.summary()
    

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/gdrive/My Drive/python-cloud/outputs/4/mnist_%d.jpg" % epoch)
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4000, batch_size=32, save_interval=50)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.870907, acc.: 28.12%] [G loss: 0.643794]
1 [D loss: 0.744922, acc.: 56.25%] [G loss: 0.818597]
2 [D loss: 0.575872, acc.: 73.44%] [G loss: 0.982314]
3 [D loss: 0.582844, acc.: 70.31%] [G loss: 1.049939]
4 [D loss: 0.494328, acc.: 84.38%] [G loss: 1.110232]
5 [D loss: 0.506541, acc.: 82.81%] [G loss: 1.118123]
6 [D loss: 0.439099, acc.: 89.06%] [G loss: 1.125846]
7 [D loss: 0.418073, acc.: 89.06%] [G loss: 1.019664]
8 [D loss: 0.442021, acc.: 84.38%] [G loss: 1.090984]
9 [D loss: 0.571291, acc.: 65.62%] [G loss: 1.224882]
10 [D loss: 0.801658, acc.: 57.81%] [G loss: 1.250025]
11 [D loss: 0.792519, acc.: 43.75%] [G loss: 1.283613]
12 [D loss: 0.785188, acc.: 51.56%] [G loss: 1.363651]
13 [D loss: 0.651239, acc.: 56.25%] [G loss: 1.096480]
14 [D loss: 0.587716, acc.: 65.62%] [G loss: 1.112109]
15 [D loss: 0.575284, acc.: 75.00%] [G loss: 0.864119]
16 [D loss: 0.493511, acc.: 76.56%] [G loss: 0.781179]
17 [D loss: 0.440219, acc.: 82.81%] [G loss: 0.681470]
18 [D loss: 0.453096